In [4]:

import pickle
import torch
import numpy as np
from pathlib import Path
def verify_pytorch_object_corrected(file_path):
    """
    Vérifie si un fichier .pkl contient un objet PyTorch valide pour GNN
    VERSION CORRIGÉE pour la vraie structure de votre convertisseur
    
    Returns:
        dict: Rapport de vérification complet
    """
    file_name = Path(file_path).name
    print(f"\n VÉRIFICATION: {file_name}")
    print("="*60)
    
    report = {
        'file_name': file_name,
        'file_exists': False,
        'loadable': False,
        'contains_pytorch': False,
        'gnn_ready': False,
        'details': {},
        'errors': []
    }
    
    # 1. Vérifier existence du fichier
    if not Path(file_path).exists():
        print(" Fichier introuvable")
        report['errors'].append("Fichier introuvable")
        return report
    
    report['file_exists'] = True
    print(" Fichier trouvé")
    
    # 2. Tenter de charger le fichier
    try:
        with open(file_path, "rb") as f:
            data = pickle.load(f)
        report['loadable'] = True
        print(" Fichier chargeable")
        print(f"   Type racine: {type(data)}")
    except Exception as e:
        print(f" Erreur chargement: {e}")
        report['errors'].append(f"Erreur chargement: {e}")
        return report
    
    # 3. Analyser la structure (VERSION CORRIGÉE)
    if isinstance(data, dict):
        print(f" Structure dict avec clés: {list(data.keys())}")
        report['details']['structure'] = 'dict'
        report['details']['keys'] = list(data.keys())
        
        # CORRECTION: Vérifier les tenseurs directement au niveau racine
        required_tensors = ['x', 'edge_index', 'edge_attr']
        tensors_found = {}
        
        print(" Recherche des tenseurs PyTorch...")
        
        for tensor_name in required_tensors:
            if tensor_name in data:
                tensor = data[tensor_name]
                if torch.is_tensor(tensor):
                    print(f"   {tensor_name}: {tensor.shape} ({tensor.dtype})")
                    tensors_found[tensor_name] = {
                        'shape': list(tensor.shape),
                        'dtype': str(tensor.dtype),
                        'device': str(tensor.device),
                        'requires_grad': tensor.requires_grad
                    }
                    report['contains_pytorch'] = True
                else:
                    print(f"    {tensor_name}: pas un tensor ({type(tensor)})")
                    report['errors'].append(f"{tensor_name} n'est pas un tensor")
            else:
                print(f"    {tensor_name}: manquant")
                report['errors'].append(f"{tensor_name} manquant")
        
        report['details']['tensors'] = tensors_found
        
        # 4. Validation GNN
        if len(tensors_found) == 3:
            print("\n VALIDATION GNN:")
            
            x = data['x']
            edge_index = data['edge_index']
            edge_attr = data['edge_attr']
            
            validation_checks = []
            
            # Check 1: Dimensions cohérentes
            num_nodes = x.size(0)
            num_edges = edge_index.size(1)
            
            validation_checks.append(('Nœuds > 0', num_nodes > 0))
            validation_checks.append(('Arêtes > 0', num_edges > 0))
            
            # Check 2: edge_index valide
            max_edge_idx = torch.max(edge_index).item()
            min_edge_idx = torch.min(edge_index).item()
            
            validation_checks.append(('Indices arêtes >= 0', min_edge_idx >= 0))
            validation_checks.append(('Indices arêtes < num_nodes', max_edge_idx < num_nodes))
            
            # Check 3: edge_index shape
            validation_checks.append(('edge_index shape [2, *]', edge_index.size(0) == 2))
            
            # Check 4: Pas de NaN
            validation_checks.append(('Pas NaN dans x', not torch.isnan(x).any()))
            validation_checks.append(('Pas NaN dans edge_attr', not torch.isnan(edge_attr).any()))
            
            # Check 5: Dimensions features
            validation_checks.append(('edge_attr nœuds = edge_index arêtes', edge_attr.size(0) == num_edges))
            
            # Check 6: Cohérence dimensions features
            validation_checks.append(('x et edge_attr même feature_dim', x.size(1) == edge_attr.size(1)))
            
            # Afficher résultats
            all_passed = True
            for check_name, passed in validation_checks:
                status = "✅" if passed else "❌"
                print(f"   {status} {check_name}")
                if not passed:
                    all_passed = False
                    report['errors'].append(f"Validation échouée: {check_name}")
            
            report['gnn_ready'] = all_passed
            report['details']['validation_checks'] = validation_checks
            
            # Statistiques
            print(f"\n STATISTIQUES:")
            print(f"   Nœuds: {num_nodes:,}")
            print(f"   Arêtes: {num_edges:,}")
            print(f"   Feature dimension: {x.size(1)}")
            print(f"   Degré moyen: {num_edges * 2 / num_nodes:.2f}")
            print(f"   Device: {x.device}")
            print(f"   Sparsité x: {torch.mean((x == 0).float()):.3f}")
            print(f"   Sparsité edge_attr: {torch.mean((edge_attr == 0).float()):.3f}")
            
            report['details']['stats'] = {
                'num_nodes': num_nodes,
                'num_edges': num_edges,
                'feature_dim': x.size(1),
                'avg_degree': num_edges * 2 / num_nodes,
                'device': str(x.device),
                'sparsity_x': float(torch.mean((x == 0).float())),
                'sparsity_edge_attr': float(torch.mean((edge_attr == 0).float()))
            }
            
        else:
            print(" Tenseurs requis manquants pour GNN")
    else:
        print(f" Structure non-dict: {type(data)}")
        report['errors'].append(f"Structure non-dict: {type(data)}")
    
    # Conclusion
    print(f"\n CONCLUSION:")
    if report['gnn_ready']:
        print("    PRÊT POUR GNN !")
    elif report['contains_pytorch']:
        print("     Contient PyTorch mais problèmes détectés")
    else:
        print("    PAS un objet PyTorch valide")
    
    return report

# %% [markdown]
##  Vérification des Fichiers avec Structure Correcte

# %%
# Chemins des fichiers à vérifier
files_to_check = [
     "../data/05_model_input/gnn_pytorch_data.pkl",
    "../data/05_model_input/baseline_gnn_pytorch_data.pkl"
]

# Vérifier chaque fichier
reports = []
for file_path in files_to_check:
    report = verify_pytorch_object_corrected(file_path)
    reports.append(report)

# %% [markdown]
##  Résumé Final

# %%
def final_summary_corrected(reports):
    """Résumé final de toutes les vérifications"""
    print("\n" + "="*80)
    print(" RÉSUMÉ FINAL - VERSION CORRIGÉE")
    print("="*80)
    
    ready_for_gnn = []
    issues_found = []
    
    for report in reports:
        file_name = report['file_name']
        
        if report['gnn_ready']:
            ready_for_gnn.append(file_name)
            print(f" {file_name}: PRÊT POUR GNN")
        else:
            issues_found.append(file_name)
            print(f" {file_name}: PROBLÈMES DÉTECTÉS")
            for error in report['errors'][:3]:  # Limiter à 3 erreurs
                print(f"   • {error}")
    
    print(f"\n BILAN:")
    print(f"   Fichiers prêts GNN: {len(ready_for_gnn)}/{len(reports)}")
    
    if len(ready_for_gnn) == len(reports):
        print("    TOUS LES FICHIERS SONT PRÊTS POUR L'ENTRAÎNEMENT GNN !")
        print("\n PROCHAINE ÉTAPE:")
        print("   Vous pouvez maintenant entraîner votre GNN avec ces données !")
        print("   Format d'utilisation:")
        print("   ```python")
        print("   import pickle")
        print("   with open('data/05_model_input/gnn_pytorch_data.pkl', 'rb') as f:")
        print("       data = pickle.load(f)")
        print("   x, edge_index, edge_attr = data['x'], data['edge_index'], data['edge_attr']")
        print("   ```")
    elif len(ready_for_gnn) > 0:
        print(f"     Seuls {ready_for_gnn} sont prêts")
    else:
        print("    AUCUN FICHIER N'EST PRÊT POUR GNN")

final_summary_corrected(reports)

# %% [markdown]
##  Test de Chargement Direct pour GNN

# %%
def test_direct_gnn_loading(file_path):
    """Test direct de chargement des tenseurs pour GNN"""
    try:
        print(f"\n TEST DIRECT: {Path(file_path).name}")
        print("-" * 40)
        
        # Chargement direct
        with open(file_path, "rb") as f:
            data = pickle.load(f)
        
        # Extraction des tenseurs
        x = data['x']
        edge_index = data['edge_index']
        edge_attr = data['edge_attr']
        
        print(f" Chargement réussi")
        print(f"   x (node features): {x.shape} {x.dtype}")
        print(f"   edge_index (connections): {edge_index.shape} {edge_index.dtype}")
        print(f"   edge_attr (edge features): {edge_attr.shape} {edge_attr.dtype}")
        
        # Test d'une opération GNN basique
        print(f"\n Test opération GNN basique:")
        
        # Test 1: Moyenne des features de nœuds
        node_mean = torch.mean(x, dim=0)
        print(f"    Moyenne node features: {node_mean.shape}")
        
        # Test 2: Indexing avec edge_index
        source_nodes = x[edge_index[0]]  # Features des nœuds source
        target_nodes = x[edge_index[1]]  # Features des nœuds target
        print(f"    Indexing source/target: {source_nodes.shape}, {target_nodes.shape}")
        
        # Test 3: Message passing simple (moyenne voisins)
        try:
            # Créer un message simple (somme source + target + edge)
            messages = source_nodes + target_nodes + edge_attr
            print(f"    Message passing basique: {messages.shape}")
        except Exception as e:
            print(f"     Message passing échoué: {e}")
        
        print(f"    {Path(file_path).name} est COMPATIBLE avec les opérations GNN !")
        return True
        
    except Exception as e:
        print(f" Erreur test direct: {e}")
        return False

# Test sur les deux fichiers
for file_path in files_to_check:
    if Path(file_path).exists():
        test_direct_gnn_loading(file_path)



 VÉRIFICATION: gnn_pytorch_data.pkl
 Fichier trouvé
 Fichier chargeable
   Type racine: <class 'dict'>
 Structure dict avec clés: ['x', 'edge_index', 'edge_attr', 'num_nodes', 'num_edges']
 Recherche des tenseurs PyTorch...
   x: torch.Size([145341, 64]) (torch.float32)
   edge_index: torch.Size([2, 678839]) (torch.int64)
   edge_attr: torch.Size([678839, 64]) (torch.float32)

 VALIDATION GNN:
   ✅ Nœuds > 0
   ✅ Arêtes > 0
   ✅ Indices arêtes >= 0
   ✅ Indices arêtes < num_nodes
   ✅ edge_index shape [2, *]
   ✅ Pas NaN dans x
   ✅ Pas NaN dans edge_attr
   ✅ edge_attr nœuds = edge_index arêtes
   ✅ x et edge_attr même feature_dim

 STATISTIQUES:
   Nœuds: 145,341
   Arêtes: 678,839
   Feature dimension: 64
   Degré moyen: 9.34
   Device: cpu
   Sparsité x: 0.915
   Sparsité edge_attr: 0.887

 CONCLUSION:
    PRÊT POUR GNN !

 VÉRIFICATION: baseline_gnn_pytorch_data.pkl
 Fichier trouvé
 Fichier chargeable
   Type racine: <class 'dict'>
 Structure dict avec clés: ['x', 'edge_index', '

In [5]:
import pickle

with open("../data/05_model_input/baseline_gnn_pytorch_data.pkl", "rb") as f:
    data = pickle.load(f)

print(type(data))
print(data.keys() if isinstance(data, dict) else data)


<class 'dict'>
dict_keys(['x', 'edge_index', 'edge_attr', 'num_nodes', 'num_edges'])
